In [1]:
import os
import json
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
ES_USER = os.getenv('ES_USER')
ES_PASSWORD = os.getenv('ES_PASSWORD')
FIRECARESID='79592'

In [43]:
es = Elasticsearch(['http://localhost:8080'], http_auth=(ES_USER,ES_PASSWORD))
                
q = Q('match',fire_department__firecares_id=FIRECARESID) & Q('range', description__event_closed={
                'lte': '2020-04-30T23:59:59-04:00','gte': '2019-05-01T00:00:00-04:00'}) & ~Q('match', address__first_due='116')
s = Search(using=es,index='*-fire-incident-*,-*apparatus*')
response = s.source(['apparatus','address.latitude','address.longitude',
                     'address.first_due','description.event_closed',
                     'description.event_opened',
                     'fire_department.firecares_id']).query(q)

In [44]:
test = [d.to_dict() for d in response.scan()]

In [46]:
len(sorted([item['description']['event_opened'] for item in test],reverse=True))

15391